# Monte Carlo simulations of the 2D Ising model

*Authors: Enze Chen (University of California, Berkeley)*

![Ising model](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/Ising_schematic.png)

This notebook guides you through the steps of setting up a Monte Carlo (MC) simulation of the 2D ferromagnetic [Ising model](https://en.wikipedia.org/wiki/Ising_model) and observing the characteristic phase transition at the critical temperature $T_c$. For more details, see the classical work by [Onsager, L. *Physical Review*, **65**, 1944](https://journals.aps.org/pr/abstract/10.1103/PhysRev.65.117). We will use a $NVT$ ensemble and the [Metroplis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm). This notebook is far from being original as several similar notebooks exist online, but I have tried to keep the code simple and explanations plentiful so that *beginners* to Python and computational MSE can successfully complete everything.

## How to run this notebook

If you are viewing this notebook on Google Colaboratory, then all the software is already set up for you (hooray). If you want to run the notebook locally, make sure all the Python libraries in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed.

For pedagogical reasons, there are a few sections for you to complete the code in order to run the simulation. These are delineated with the dashed lines as follows, and you should **only change what's inside**. You don't have to edit the text or code anywhere else. I've also included "**TODO**" to separate the background context from the actual instructions.
```python
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #
```
If you edit the code in a cell, just press `Shift+Enter` to run it again. You have to execute **all** the code cells in this notebook from top to bottom (so don't skip around). A number `[#]` will appear to the left of the code cell once it's done executing. When all done successfully, you should be able to see a few images of your system and plots of the system properties as a function of $T$.

## Acknowledgements

I thank [Dr. Matthew Sherburne](https://mse.berkeley.edu/people_new/sherburne/) for teaching MATSCI 215: Computational Materials Science and my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for encouraging me in my education-related pursuits. An interactive version of this notebook can be found online at [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/Monte_Carlo_Ising_model.ipynb). 

## Important equations

The [Ising model](https://en.wikipedia.org/wiki/Ising_model) is one of the simplest models for magnetism where the magnetic dipole moments ("spins") are discrete values from the set $\{+1, -1\}$ corresponding to `up` and `down` spins, respectively. We will work with a 2D square lattice, so the spin at location $(i,j)$ will be denoted by $\sigma_{ij}$. The 2D square lattice Ising model is also one of the simplest systems to display a *phase transition*.

The first equation we must write down is the **Hamiltonian**, which in our case will only feature an **exchange interaction** parameter $J$ that is positive for ferromagnetism (we ignore the magnetic field interaction parameter $h$). Furthermore, this exchange interaction only applies to nearest-neighbors where we will also assume **periodic boundary conditions**. Therefore,

$$ H(\sigma) = -J \sum_{\langle p,q \rangle} \sigma_{p_i,p_j} \sigma_{q_i,q_j}$$

where $\langle p,q \rangle$ represent a summation over adjacent lattice sites only. We'll be using the [Metroplis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) for the MC simulation, which has the following steps for each iteration through the cycle. As we begin coding, we'll find ways of optimizing the calculations.

1. Choose a site $(i,j)$ at random.
1. Calculate the change in energy, $\Delta E$, if the sign at that site was flipped.
1. If $\Delta E \le 0$, accept the change by keeping the new spin because we've lowered the total energy of our system.
1. If $\Delta E > 0$, generate a random number $r \in [0, 1]$.
    1. Accept the change only if $r < \exp (-\beta \Delta E)$ where $\beta = \frac{1}{k_BT}$ and $k_B$ is the Boltzmann constant.
    1. Otherwise, return the spin to the original state.
    
Since our goal is to simulate a phase transition, there must be properties of the system that we can track. The properties that are relevant to us are the expected values of the system's **energy** ($E$) and **magnetization** ($M$), where the expected value of a quantity $Q$ over **both space and time** is given by 

$$ \langle Q \rangle_{N,T} = \frac{1}{T} \sum_{t=1}^{T} \left[ \frac{1}{N} \sum_{s=1}^{N} Q_{s,t} \right] $$

Knowing the expected values will allow us to calculate the **heat capacity** per site and **magnetic susceptibility** per site, which are given respectively by

$$ C = \frac{\langle E^2 \rangle - \langle E \rangle^2}{k_B T^2 L^2} $$ 

$$ \chi = \frac{\langle M^2 \rangle - \langle M \rangle^2}{k_B T L^2} $$

where $L$ is the size of one dimension. At the phase transition, we should see a divergence in these two quantities (which might not be perfect due to finite size effects).

## Known issues

* As the code isn't heavily optimized, it will be slow if you run it for too many iterations or for too large of a system. Please be gentle. ❤

## Python library imports

These are all the required Python libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Normalized energies

To make our lives even simpler, we will work with dimensionless units, which can be achieved by setting $J = 1$ and $k_B = 1$. In the subsequent code, we'll still include $J$ and $k_B$ in the appropriate places to be as general as possible.

**TODO**: Set the correct constants below.

In [ ]:
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #

## Helper functions

We'll write a few helper functions that we'll call in the main body. For this first one, `create_spins()` will initialize the $L \times L$ grid of spins randomly.

**TODO**: Complete the function below which will create an $L \times L$ array of spins randomly selected from the set $\{+1, -1\}$. You might find the [`np.random.choice()`](https://numpy.org/devdocs/reference/random/generated/numpy.random.choice.html) helpful.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def create_spins(L):
    '''Create an L x L array of spins randomly chosen from {+1, -1}.
    
    Args:
        L (int): The number of lattice sites along each dimension.
    
    Returns:
        A NumPy array of spins of dimension L x L.
    '''
    pass   # delete this and write your own code
# ---------------------- #

No code is complete without tests, so we'll test our function below with an easy case. I suggest running the following cell at least twice to make sure your spins are randomly being assigned.

In [ ]:
# Test for the create_spins() function
create_spins(3)

### Plotting utility

Next we'll write a helper function that assists with plotting the spins. This part can be safely skipped, but completing it can help you visualize what's going on in your system. Some hints:
* We can use the [`ax.imshow()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html) function to display the spins as a heatmap.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
plt.rcParams.update({'figure.figsize':(4,4), 'font.size':14, 'image.cmap':'coolwarm'})
def plot_spins(spins, T):
    """Display the spin configurations in your system.
    
    Args:
        spins (numpy.ndarray): An array of spin values.

    Returns:
        None, but a pyplot is displayed.
    """
    pass   # delete this and write your own code
# ---------------------- #
# We'll test it here
plot_spins(create_spins(16), T=1)

### Total energy and magnetization

Since we care about the total energy and total magnetization in our system, let's write helper functions to help us calculate those quantities.

**TODO**: Finish the `compute_energy()` and `compute_mag()` functions to calculate the total energy and total magnetization, respectively, which are given by:

$$ E = -J \sum_{\langle p,q \rangle} \sigma_{p_i,p_j} \sigma_{q_i,q_j} \qquad M = \sum_{i,j} \sigma_{ij}$$

Some hints:

* Each lattice site $(i,j)$ has four nearest neighbors given by the following figure. However you do the sum, make sure to account for overcounting in your final answer for the energy.

![Ising neighbors](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/Ising_neighbors.png)

* You can index into a 2D NumPy array at site $(i,j)$ with the syntax `spins[i, j]`.

* Since we are using periodic boundary conditions, you might find the [modulo operator](https://www.tutorialspoint.com/What-is-modulo-operator-in-Python) `%` handy.

* Calculating the total magnetization should be very simple: you just have to add up all the spins in your array. Check out [`np.sum()`](https://numpy.org/doc/1.18/reference/generated/numpy.sum.html).

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def compute_energy(spins):
    '''Compute the total energy in the Ising model.
    
    Args:
        spins (numpy.ndarray): An array of spin values.
    
    Returns:
        A float for the total energy.    
    '''
    pass   # delete this and write your own code

def compute_mag(spins):
    '''Compute the total magnetization in the Ising model.
    
    Args:
        spins (numpy.ndarray): An array of spin values.
        
    Returns:
        A float for the total magnetization.
    '''
    pass   # delete this and write your own code
# ---------------------- #

Once again, we'll write some tests for known cases. I encourage you to write some more! My `print()` statements are using [f-strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) which are new as of Python 3.6 and wicked cool.

In [ ]:
# Tests for compute_energy() and compute_mag()
spins_all_pos = np.array([[1, 1], [1, 1]])  # should return E = -4 and M = 4
print(f'The all positive case has a total energy of {compute_energy(spins_all_pos)} ' + 
      f'and a total magnetization of {compute_mag(spins_all_pos)}.')
spins_alt = np.array([[1, -1], [-1, 1]])    # should return E = 4 and M = 0
print(f'The alternating spins case has a total energy of {compute_energy(spins_alt)} ' + 
      f'and a total magnetization of {compute_mag(spins_alt)}.')

### Monte Carlo sweeps

The next helper function will perform one MC sweep, which means we'll choose a random site within our lattice and decide if we should flip the spin or not. We must do this $L^2$ times.

**TODO**: Finish the `mc_sweep()` function below. Some hints are:
* You might find the [`np.random.randint()`](https://numpy.org/devdocs/reference/random/generated/numpy.random.randint.html) method helpful.
* Note that while we could compute the total energy twice, we only really need the energy *difference* $\Delta E$. Furthermore, our system is discretized nicely such that the spins only take on $+1$ and $-1$. Can we leverage this somehow?
* Don't forget we are using the Metroplis-Hastings algorithm, so you might need functions like `np.random.random()` and `np.exp()`.
* Let's be cognizant of memory usage and make this changes to `spins` in-place (i.e. don't make excessive copies of the `spins` array).

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def mc_sweep(spins, beta):
    '''Peform one MC sweep through all the sites.
    
    Args:
        spins (numpy.ndarray): An array of spin values.
        beta (float): Inverse temperature.
        
    Returns:
        A NumPy array of spins.
    '''
    pass   # delete this and write your own code
# ---------------------- #

While we can't necessarily control which sites the MC algorithm will select, we can at least see if our code above is flipping signs at all using the following test code.

In [ ]:
orig = create_spins(5)
dup = np.copy(orig)  # we have to make a copy because our changes are in-place
mc_sweep(spins=dup, beta=1)
print(f'The spins changed at {np.count_nonzero(dup - orig)} sites!')

## Main method

Now that we have created all the pieces, it's time to assemble them. 

**TODO**: We need to write the main method, which we will call `mc_ising_model()`. Read the docstring for what the input arguments correspond to. Some hints:
* Don't forget to include an equilibration period!
* We should also average of properties over the number of `mcsteps` and normalize by the system size to get an intensive quantity.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def mc_ising_model(L, Ts, eqsteps, mcsteps):
    '''Perform an MC simulation for the 2D ferromagnetic Ising model
       and calculate relevant physical properties.
    
    Args:
        L (int): Number of lattice sites along one side.
        Ts (numpy.ndarray): A list of temperatures to simulate.
        eqsteps (int): Number of equilibration MC steps.
        mcsteps (int): Number of additional MC steps.
        
    Returns:
        E_T (list): The average energy at each temperature.
        M_T (list): The average magnetization at each temperature.
        C_T (list): The average heat capacity at each temperature.
        X_T (list): The average susceptibility at each temperature.
    '''
    
    # Store the final values as a function of temperature
    E_T = []
    M_T = []
    C_T = []
    X_T = []
    
    # Return the four lists as a tuple
    return (E_T, M_T, C_T, X_T)
# ---------------------- #

Next we need to specify the parameters of our MC simulation, as described in the docstring above. Note that to start off, you might want to make all these values small to make sure your code runs correctly before using larger values for more accurate statistics.

**TODO**: Set the correct experimental parameters below. Again, I wouldn't recommend anything too crazy.

In [ ]:
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #
E_T, M_T, C_T, X_T = mc_ising_model(L=L, Ts=Ts, eqsteps=eqsteps, mcsteps=mcsteps)

**TODO**: Now we will plot the properties we obtained above as a function of $T$ and estimate $T_c$ based on our simulations. Hopefully your estimated $T_c$ and transitions are near the theoretical value of:

$$ T_c = \frac{2J}{k_B \ln \left( 1 + \sqrt{2} \right)} \approx 2.269 $$

Some hints:
* You'll want to reference the variables you used previously.
* You can plots on different axes from the subplots using `ax[i].plot()` where `i` is the index.
* For the magnetization, it might be easier for you to plot the absolute value. You can obtain the absolute value with [`np.absolute()`](https://numpy.org/doc/1.18/reference/generated/numpy.absolute.html),
* You can set the axis title with the function `ax[i].set_title('Title')` and $x$-axis label with `ax[i].set_xlabel('Label')`.
* There are many ways of estimating the transition temperature. One way is to average the $T$s at which $C$ and $\chi$ are maximized. You might find [`np.argmax()`](https://numpy.org/doc/1.18/reference/generated/numpy.argmax.html) handy.

In [ ]:
plt.rcParams.update({'figure.figsize':(16,3), 'lines.markersize':8})
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #

How do the results change as you change $L$ and `eqsteps`?

----------------

## Conclusion

I hope you learned how we can use Monte Carlo and the Metropolis-Hastings algorithm to simulate the 2D Ising model and observe the characteristic phase transition. If you have any remaining questions or ideas for this and other modules, please don't hesitate to reach out.


## Extensions

* One of the benefits of MC sampling is the ability to incorporate uncertainties into the calculations. Can you add this into the `mc_ising_model()` function above? You can then display the uncertainties with the [`plt.errorbar()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.errorbar.html) function.

* Can you modify the code to calculate other lattice geometries (e.g. triangular)?

* Can you compute a radial distribution function?


## Answers

If you found yourself stuck at certain points, I provide some sample answers [here](https://github.com/enze-chen/learning_modules/blob/master/data/answers.md#Monte_Carlo_Ising_model).